In [17]:
import numpy as np
import pandas as pd

# Huffman encoder implementation

In [18]:
# this block of code produces the huffman code (a dictionary of symbols and codewords)

class Node:
    def __init__(self, value):

        self.value = value
        self.left = None
        self.right = None
        

def binary_tree(dictionary,verbose):
    N = len(dictionary)
    alphabet = [i for i in dictionary.keys()]
    dictionary_sorted = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True)) 
    leaves = list(dictionary_sorted.items())
    child_id=[]                       

    if verbose:
        print("*** binary tree *** \n")
        print("* leaves:")
        print(leaves)
        print("----------------------------------------------------------------------------")

    for i in range(N-1):
        child_sx_id = leaves[-2][0]
        child_dx_id = leaves[-1][0]
        child_sx_value = leaves[-2][1]
        child_dx_value = leaves[-1][1]  

        parent_name = f"P{i}"
        parent_value = child_sx_value + child_dx_value
        parent_leaf = [(parent_name, parent_value)]

        child_id.append([parent_name, child_sx_id, child_dx_id])

        if verbose:
            print("+ parent leaf:")
            print(parent_leaf, "<--", child_sx_id, "+", child_dx_id, "\n")

        leaves = leaves[:-2] + parent_leaf
        leaves = sorted(leaves, key=lambda x: x[1], reverse=True)

        if verbose:
            if len(leaves) != 1:
                print("* leaves:")
            else:
                print("++ root:")
            print(leaves)
            print("----------------------------------------------------------------------------")

    return alphabet, child_id


def build_huffman_tree(parent_child_relations):
    nodes = {}
    
    for relation in parent_child_relations:
        parent_val, left_val, right_val = relation
        parent_node = nodes.get(parent_val, Node(parent_val))
        left_node = nodes.get(left_val, Node(left_val))
        right_node = nodes.get(right_val, Node(right_val))
        
        parent_node.left = left_node
        parent_node.right = right_node
        
        nodes[parent_val] = parent_node
        nodes[left_val] = left_node
        nodes[right_val] = right_node
    
    return nodes[parent_val]


def generate_huffman_codes(root, code='', codes={}):
    if root:
        if root.value != "":
            codes[root.value] = code
        generate_huffman_codes(root.left, code + "0", codes)
        generate_huffman_codes(root.right, code + "1", codes)

    return codes


def huffman_codes(dictionary,verbose=False):
    alphabet, child_id = binary_tree(dictionary,verbose=verbose)
    huffman_tree = build_huffman_tree(child_id)
    huffman_codes = generate_huffman_codes(huffman_tree)
    
    huffman_codes = {key: value for key, value in huffman_codes.items() if key in alphabet}
    huffman_codes = {key: huffman_codes[key] for key in alphabet}

    if verbose:
        print("\ncodewords:")
        print(huffman_codes)

    return huffman_codes    

In [19]:
# auxiliary function that evaluetes probabilities of symbols from a strig:

def generate_symbol_frequency(input):
    if len(input)==1:
        text=input[0]
    else: text=input    
    
    symbol_frequency = {}
    total_symbols = len(text)

    for char in text:
        if char in symbol_frequency:
            symbol_frequency[char] += 1
        else:
            symbol_frequency[char] = 1

    for char in symbol_frequency:
        symbol_frequency[char] /= total_symbols

    return symbol_frequency


# auxiliary function that given a string and the huffman codewords encodes the string

def string_encoder(input, dictionary):
    encoded_string=[]

    if len(input)==1:
        string=input[0]

    else:
        string=input

    for i in string:
        for j in dictionary.keys():
            if i==j:
                encoded_string.append(dictionary[j])

    encoded_string=''.join(encoded_string)

    return encoded_string

In [20]:
# final huffman encoder

def huffman_encoder(string, verbose=False):
    alphabet=generate_symbol_frequency(string)
    codewords=huffman_codes(alphabet)
    encoded_string=string_encoder(string,codewords)

    if verbose:
        print(codewords)

    return encoded_string, codewords    

In [21]:
# huffman decoder

def huffman_decoder(binary_string, codewords):
    reverse_codewords = {i: j for j, i in codewords.items()}
    decoded_message = ''
    current_code = ''

    for bit in binary_string:
        current_code += bit
        if current_code in reverse_codewords:
            decoded_message += reverse_codewords[current_code]
            current_code = ''  

    return decoded_message

##  Encoder test:

Let's start with a simple dictionary of symbols and frequencies and build up the Huffman codewords from the binary tree.

In [22]:
# fist test

dict_test={"a":0.05,"b":0.2,"c":0.35,"d":0.4}
codewords_test=huffman_codes(dict_test,verbose=True)

*** binary tree *** 

* leaves:
[('d', 0.4), ('c', 0.35), ('b', 0.2), ('a', 0.05)]
----------------------------------------------------------------------------
+ parent leaf:
[('P0', 0.25)] <-- b + a 

* leaves:
[('d', 0.4), ('c', 0.35), ('P0', 0.25)]
----------------------------------------------------------------------------
+ parent leaf:
[('P1', 0.6)] <-- c + P0 

* leaves:
[('P1', 0.6), ('d', 0.4)]
----------------------------------------------------------------------------
+ parent leaf:
[('P2', 1.0)] <-- P1 + d 

++ root:
[('P2', 1.0)]
----------------------------------------------------------------------------

codewords:
{'a': '011', 'b': '010', 'c': '00', 'd': '1'}


In [23]:
# second test:

P_h=9/10
P_t=1/10

df=pd.DataFrame({"Events":["h","t"],"Probabilities":[P_h,P_t]})
print(df,"\n")

p_hhh=df["Probabilities"][0]**3
p_hht=p_hth=p_thh=df["Probabilities"][0]*df["Probabilities"][0]*df["Probabilities"][1]
p_htt=p_tht=p_tth=df["Probabilities"][0]*df["Probabilities"][1]*df["Probabilities"][1]
p_ttt=df["Probabilities"][1]*df["Probabilities"][1]*df["Probabilities"][1]

df_3=pd.DataFrame({"Events":["hhh","hht","hth","thh","htt","tht","tth","ttt"],"Probabilities":[p_hhh,p_hht,p_hth,p_thh,p_htt,p_tht,p_tth,p_ttt]})
print(df_3,"\n")

dict_test_2={}
for i,j in zip(df_3["Events"],df_3["Probabilities"]):
    dict_test_2[i]=j

codewords_test_2=huffman_codes(dict_test_2)
print("codewords:")
print(codewords_test_2)

  Events  Probabilities
0      h            0.9
1      t            0.1 

  Events  Probabilities
0    hhh          0.729
1    hht          0.081
2    hth          0.081
3    thh          0.081
4    htt          0.009
5    tht          0.009
6    tth          0.009
7    ttt          0.001 

codewords:
{'hhh': '0', 'hht': '100', 'hth': '101', 'thh': '110', 'htt': '11100', 'tht': '11101', 'tth': '11110', 'ttt': '11111'}


Now let's start directly from a string and see if the code is able to encode the original string into a binary one.

In [24]:
# now let's try to encode a given string directly:

print("original string:")
message='sadlasdiikkliedscc'
print(message,"\n")

print("symbols frequencies:")
symbl_freq=generate_symbol_frequency(message)
print(symbl_freq,"\n")

print("Huffman codewords:")
encoded_message, __ =huffman_encoder(message,verbose=True)

print("\nencoded string:")
print(encoded_message)

original string:
sadlasdiikkliedscc 

symbols frequencies:
{'s': 0.16666666666666666, 'a': 0.1111111111111111, 'd': 0.16666666666666666, 'l': 0.1111111111111111, 'i': 0.16666666666666666, 'k': 0.1111111111111111, 'e': 0.05555555555555555, 'c': 0.1111111111111111} 

Huffman codewords:
{'s': '11', 'a': '011', 'd': '000', 'l': '100', 'i': '001', 'k': '101', 'e': '0101', 'c': '0100'}

encoded string:
110110001000111100000100110110110000101010001101000100


In [25]:
# second test on a full string:

print("original string:")
message_2='hello world!'
print(message_2,"\n")

print("symbols frequencies:")
symbl_freq_2=generate_symbol_frequency(message_2)
print(symbl_freq_2,"\n")

print("Huffman codewords:")
encoded_message_2, codewords_2=huffman_encoder(message_2,verbose=True)

print("\nencoded string:")
print(encoded_message_2)

original string:
hello world! 

symbols frequencies:
{'h': 0.08333333333333333, 'e': 0.08333333333333333, 'l': 0.25, 'o': 0.16666666666666666, ' ': 0.08333333333333333, 'w': 0.08333333333333333, 'r': 0.08333333333333333, 'd': 0.08333333333333333, '!': 0.08333333333333333} 

Huffman codewords:
{'h': '101', 'e': '1000', 'l': '01', 'o': '11', ' ': '1001', 'w': '0010', 'r': '0011', 'd': '0000', '!': '0001'}

encoded string:
1011000010111100100101100110100000001


## Decoder test:
Now our final test is try to decode the encoded message: 

In [26]:
print("\nencoded string:")
print(encoded_message_2,"\n")

decoded_message_2=huffman_decoder(encoded_message_2,codewords_2)
print("decoded string:")
print(decoded_message_2)


encoded string:
1011000010111100100101100110100000001 

decoded string:
hello world!


## Final test: encode the incipit of Dante's Inferno

In [27]:
inferno = 'Nel mezzo del cammin di nostra vita \nmi ritrovai per una selva oscura, \nché la diritta via era smarrita.'
print(inferno)


Nel mezzo del cammin di nostra vita 
mi ritrovai per una selva oscura, 
ché la diritta via era smarrita.


In [28]:
encoded_inferno, inferno_codewords = huffman_encoder([inferno])
inferno_codewords

{'N': '0010010',
 'e': '1000',
 'l': '01000',
 ' ': '000',
 'm': '1011',
 'z': '010100',
 'o': '01001',
 'd': '01011',
 'c': '10100',
 'a': '011',
 'i': '110',
 'n': '10101',
 's': '00110',
 't': '1001',
 'r': '111',
 'v': '00111',
 '\n': '010101',
 'p': '0010011',
 'u': '001010',
 ',': '0010000',
 'h': '0010001',
 'é': '0010110',
 '.': '0010111'}

In [29]:
encoded_inferno

'0010010100001000000101110000101000101000100100001011100001000000101000111011101111010101000010111100001010101001001101001111011000001111101001011000010101101111000011111010011110100100111011110000001001110001110000010101010101100000110100001000001110110000100100110101000010101110110010000000010101101000010001001011000001000011000010111101111101001100101100000111110011000100011101100000110101101111111111010010110010111'

In [30]:
decoded_inferno=huffman_decoder(encoded_inferno,inferno_codewords)
print(decoded_inferno)

Nel mezzo del cammin di nostra vita 
mi ritrovai per una selva oscura, 
ché la diritta via era smarrita.
